In [1]:
import dask
import numpy as np

import skimage
from skimage import data, img_as_float, restoration
from skimage.filters import rank
from skimage.util import apply_parallel

img = img_as_float(data.camera())
img = np.tile(img, (4, 8))
sigma = 0.15
rstate = np.random.RandomState(1234)
img = img + sigma * rstate.standard_normal(img.shape)

chunks = data.camera().shape

sigma_est = restoration.estimate_sigma(img)

# Note: depending on the scikit-image version used, may have to change from `multichannel=False` to `channel_axis=None`.

if np.lib.NumpyVersion(skimage.__version__) < '0.19.0dev0':
    channel_kwarg = {'multichannel': False}
else:
    channel_kwarg = {'channel_axis': None}

In [2]:
extra_keywords = dict(weight=0.1, n_iter_max=25)
extra_keywords.update(channel_kwarg)
print("\nBenchmarking restoration.denoise_tv_chambolle (with apply_parallel)")
%timeit dn_tv_c = apply_parallel(restoration.denoise_tv_chambolle, img, chunks=chunks, depth=16, extra_keywords=extra_keywords, dtype=img.dtype, compute=True, **channel_kwarg)
print("Benchmarking restoration.denoise_tv_chambolle (without apply_parallel)")
%timeit restoration.denoise_tv_chambolle(img, **extra_keywords)



Benchmarking restoration.denoise_tv_chambolle (with apply_parallel)
2.68 s ± 56.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Benchmarking restoration.denoise_tv_chambolle (without apply_parallel)
8.02 s ± 166 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [2]:
extra_keywords = dict(weight=1, max_iter=25)  # , multichannel=False)
print("\nBenchmarking restoration.denoise_tv_bregman (with apply_parallel)")
%timeit dn_tv_b = apply_parallel(restoration.denoise_tv_bregman, img, chunks=chunks, depth=16, extra_keywords=extra_keywords, dtype=img.dtype, compute=True)
print("Benchmarking restoration.denoise_tv_bregman (without apply_parallel)")
%timeit restoration.denoise_tv_bregman(img, **extra_keywords)

# Note on scikit-image <= 0.18.x this will give a slow result due to
# https://github.com/scikit-image/scikit-image/pull/5400 


Benchmarking restoration.denoise_tv_bregman (with apply_parallel)
509 ms ± 37.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Benchmarking restoration.denoise_tv_bregman (without apply_parallel)
2.35 s ± 19.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
patch_distance = 5
patch_size = 5
depth = patch_distance + patch_size // 2
extra_keywords = dict(patch_size=5, patch_distance=5, h=0.4*sigma, sigma=sigma, fast_mode=True, preserve_range=False, **channel_kwarg)
extra_keywords.update(channel_kwarg)
print("\nBenchmarking restoration.denoise_nl_means (with apply_parallel)")
%timeit dn_nlm = apply_parallel(restoration.denoise_nl_means, img, chunks=chunks, depth=depth, extra_keywords=extra_keywords, dtype=img.dtype, compute=True, **channel_kwarg)
print("\nBenchmarking restoration.denoise_nl_means (without apply_parallel)")
%timeit restoration.denoise_nl_means(img, **extra_keywords)



Benchmarking restoration.denoise_nl_means (with apply_parallel)
1.22 s ± 29.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

Benchmarking restoration.denoise_nl_means (without apply_parallel)
8.06 s ± 51.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [3]:
print("\nBenchmarking restoration.denoise_bilateral")
img_clip = np.clip(img, 0, 1)
extra_keywords = {}
print("\nBenchmarking restoration.denoise_nl_means (with apply_parallel)")
%timeit dn_bilateral = apply_parallel(restoration.denoise_bilateral, img_clip, chunks=chunks, depth=8, extra_keywords=extra_keywords, dtype=img.dtype, compute=True)
print("\nBenchmarking restoration.denoise_nl_means (without apply_parallel)")
%timeit restoration.denoise_bilateral(img_clip, **extra_keywords)

# Note on scikit-image <= 0.18.x this will give a slow result due to
# https://github.com/scikit-image/scikit-image/pull/5400 


Benchmarking restoration.denoise_bilateral

Benchmarking restoration.denoise_nl_means (with apply_parallel)
779 ms ± 86.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

Benchmarking restoration.denoise_nl_means (without apply_parallel)
3.75 s ± 32.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
extra_keywords = dict(sigma=sigma_est, wavelet='sym4', wavelet_levels=3)
extra_keywords.update(channel_kwarg)
print("\nBenchmarking restoration.denoise_wavelet (with apply_parallel)")
%timeit dn_wav = apply_parallel(restoration.denoise_wavelet, img, chunks=chunks, depth=8, extra_keywords=extra_keywords, dtype=img.dtype, compute=True, **channel_kwarg)
print("Benchmarking restoration.denoise_wavelet (without apply_parallel)")
%timeit restoration.denoise_wavelet(img, sigma=sigma_est, wavelet='sym4', wavelet_levels=3, **channel_kwarg)



Benchmarking restoration.denoise_wavelet (with apply_parallel)


/home/lee8rx/mambaforge/envs/skimage_dev/lib/python3.9/site-packages/pywt/_multilevel.py:43: UserWarning: Level value of 3 is too high: all coefficients will experience boundary effects.
  warnings.warn(
/home/lee8rx/mambaforge/envs/skimage_dev/lib/python3.9/site-packages/pywt/_multilevel.py:43: UserWarning: Level value of 3 is too high: all coefficients will experience boundary effects.
  warnings.warn(
/home/lee8rx/mambaforge/envs/skimage_dev/lib/python3.9/site-packages/pywt/_multilevel.py:43: UserWarning: Level value of 3 is too high: all coefficients will experience boundary effects.
  warnings.warn(
/home/lee8rx/mambaforge/envs/skimage_dev/lib/python3.9/site-packages/pywt/_multilevel.py:43: UserWarning: Level value of 3 is too high: all coefficients will experience boundary effects.
  warnings.warn(
/home/lee8rx/mambaforge/envs/skimage_dev/lib/python3.9/site-packages/pywt/_multilevel.py:43: UserWarning: Level value of 3 is too high: all coefficients will experience boundary effect

467 ms ± 13.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Benchmarking restoration.denoise_wavelet (without apply_parallel)
858 ms ± 31.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
selem = np.ones((5, 5), dtype=np.uint8)
depth = selem.shape[0] // 2
img_uint = (np.clip(img, 0, 1) * 255).astype(np.uint8)
extra_keywords = dict(selem=selem)
print("\nBenchmarking filters.rank.median (with apply_parallel)")
%timeit rank_median = apply_parallel(rank.median, img_uint, chunks=chunks, depth=depth, extra_keywords=extra_keywords, dtype=img.dtype, compute=True)
print("\nBenchmarking filters.rank.median (without apply_parallel)")
%timeit rank.median(img_uint, **extra_keywords)



Benchmarking filters.rank.median (with apply_parallel)
311 ms ± 16.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

Benchmarking filters.rank.median (without apply_parallel)
1.64 s ± 21.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
